Plan (pseudocode, detailed):
- Add small config parameters to tighten recognition:
  - recognition_min_votes (total votes across short time window)
  - recognition_distance_threshold (strict distance to auto-confirm)
  - recognition_consecutive_frames (same name consecutive frames to confirm)
  - recognition_time_window (seconds to consider recent votes)
- Add imports needed: time and collections.defaultdict.
- For each detected person bbox produce a coarse person_id by quantizing bbox center (reduce sensitivity to small bbox jitter).
- Provide helper methods:
  - _get_person_id(bbox) -> str: quantize center to grid cell id.
  - _update_detection_history(person_id, name, distance): update per-person vote counts, consecutive count, last_update.
  - _confirm_recognition(person_id, name, distance) -> bool: decide if known using votes, consecutive frames, or strict distance. Reset stale history beyond time window.
  - confirm_face(face_encoding, bbox) -> (name_or_None, confirmed:bool, distance): compute distances, choose best name or UNKNOWN, update history and return confirmation.
- Keep public API minimal so existing processing loop can call confirm_face for each candidate face and only trigger alerts when confirmed==True.
- Preserve existing logging and cooldown logic; only trigger email/alarm when confirm_face returns confirmed True.
- Provide clear defaults that are conservative (reduce false positives): require at least 2 votes or 2 consecutive frames or very close distance.

Code (patch to security_system.py):


In [1]:
# ...existing code...
import os
import json
import time
import logging
from collections import defaultdict
from datetime import datetime
from dataclasses import dataclass
from pathlib import Path
from typing import List, Tuple, Optional, Dict

import cv2
import numpy as np
import face_recognition
import mediapipe as mp
from ultralytics import YOLO
import smtplib
import pygame
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# initialize module logger
logger = logging.getLogger(__name__)
# If not configured elsewhere, configure basic logging to avoid silent failures during development
if not logging.getLogger().handlers:
    logging.basicConfig(level=logging.INFO)

@dataclass
class SecurityConfig:
    """Configuration class for security system settings."""
    known_faces_dir: str = "family_members"
    alarm_sound_path: str = "./pols-aagyi-pols.mp3"
    alarm_cooldown: int = 3
    confidence_threshold: float = 0.5
    face_recognition_tolerance: float = 0.6
    pose_visibility_threshold: float = 0.5
    camera_index: int = 0
    fallback_video: Optional[str] = "security_output.avi"
    email_enabled: bool = False
    smtp_server: str = "smtp.gmail.com"
    smtp_port: int = 587
    sender_email: str = ""
    sender_password: str = ""
    recipient_email: str = ""
    # New stricter recognition controls to reduce false positives
    recognition_min_votes: int = 2                 # total votes in time window to accept
    recognition_distance_threshold: float = 0.45   # very strict distance to auto-accept
    recognition_consecutive_frames: int = 2        # same name in consecutive frames to accept
    recognition_time_window: float = 3.0           # seconds to keep voting history
# ...existing code...

class SecuritySystem:
    """Main security system class."""

    def __init__(self, config: SecurityConfig):
        self.config = config
        # typed lists help readability but are not required at runtime
        self.known_face_encodings: List[np.ndarray] = []
        self.known_face_names: List[str] = []
        self.last_alarm_time: float = 0.0
        # detection_history mapping person_id -> metadata
        self.detection_history: Dict[str, Dict] = {}
        self.frame_count: int = 0
        self.sound_loaded: bool = False

        # Initialize components (methods must be implemented elsewhere in the class/file)
        self._initialize_models()
        self._load_known_faces()
        self._setup_audio()
        self._setup_camera()

        # detection_history structure:
        # { person_id: { 'name_counts': defaultdict(int), 'last_name': str, 'consecutive': int, 'last_update': float } }
        self.detection_history = {}
# ...existing code...

    def has_capture(self) -> bool:
        """Return True if a video capture is available and opened."""
        return hasattr(self, 'cap') and self.cap is not None and self.cap.isOpened()

    # --- Robust recognition helpers to reduce false positives ---
    def _get_person_id(self, bbox: Tuple[int, int, int, int]) -> str:
        """Compute a coarse person id from bbox center quantized to reduce jitter."""
        x1, y1, x2, y2 = bbox
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2
        # Quantize center to 50-pixel grid to tolerate small movement/jitter
        return f"{cx // 50}_{cy // 50}"

    def _update_detection_history(self, person_id: str, name: str, distance: float) -> None:
        """Update per-person recent votes and consecutive counts."""
        now = time.time()
        entry = self.detection_history.get(person_id)
        if entry is None:
            entry = {
                'name_counts': defaultdict(int),
                'last_name': None,
                'consecutive': 0,
                'last_update': now
            }
            self.detection_history[person_id] = entry

        # Reset history if stale
        if now - entry['last_update'] > self.config.recognition_time_window:
            entry['name_counts'] = defaultdict(int)
            entry['last_name'] = None
            entry['consecutive'] = 0

        # Tally vote
        entry['name_counts'][name] += 1

        # Update consecutive counting
        if entry['last_name'] == name:
            entry['consecutive'] += 1
        else:
            entry['last_name'] = name
            entry['consecutive'] = 1

        entry['last_update'] = now

    def _confirm_recognition(self, person_id: str, name: str, distance: float) -> bool:
        """Decide whether the name is confirmed for the person_id using votes / consecutive frames / strict distance."""
        now = time.time()
        entry = self.detection_history.get(person_id)
        if not entry:
            return False

        # if stale, require fresh votes
        if now - entry['last_update'] > self.config.recognition_time_window:
            return False

        # Quick accept if distance is very small (very confident)
        if name != "UNKNOWN" and distance <= self.config.recognition_distance_threshold:
            return True

        # Accept if enough votes in time window
        votes = entry['name_counts'].get(name, 0)
        if name != "UNKNOWN" and votes >= self.config.recognition_min_votes:
            return True

        # Accept if same name seen consecutively sufficient frames
        if name != "UNKNOWN" and entry['consecutive'] >= self.config.recognition_consecutive_frames:
            return True

        return False

    def confirm_face(self, face_encoding: np.ndarray, bbox: Tuple[int, int, int, int]) -> Tuple[Optional[str], bool, Optional[float]]:
        """
        Public helper: compute best-match name and update/confirm recognition robustly.
        Returns: (name_or_None, confirmed: bool, distance_or_None)
        - name_or_None: known person's name when confirmed, None otherwise
        - confirmed: True when recognition meets stricter acceptance criteria
        - distance_or_None: distance to best match (lower is better) or None if no known encodings
        """
        if not self.known_face_encodings:
            return None, False, None

        try:
            distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
        except Exception as e:
            logger.error(f"face_distance failed: {e}")
            return None, False, None

        # Ensure distances is iterable and non-empty
        if len(distances) == 0:
            return None, False, None

        best_idx = int(np.argmin(distances))
        best_dist = float(distances[best_idx])
        # Candidate name: accept only if within loose tolerance, else mark UNKNOWN
        candidate_name = self.known_face_names[best_idx] if best_dist <= self.config.face_recognition_tolerance else "UNKNOWN"

        person_id = self._get_person_id(bbox)
        self._update_detection_history(person_id, candidate_name, best_dist)
        confirmed = self._confirm_recognition(person_id, candidate_name, best_dist)

        if confirmed and candidate_name != "UNKNOWN":
            return candidate_name, True, best_dist
        return None, False, best_dist
    # --- end robust recognition helpers ---



pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
